In [2]:
import pandas as pd
import numpy as np
import os
import ast
from collections import Counter
import pickle
from ast import literal_eval
from collections import defaultdict

d:\Anaconda\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# Recipe Data From Kaggle.com

os.chdir(r"D:\recipe_recommender\recipe_recommender_webiste\food-com-recipes-and-user-interactions")
interactions_train = pd.read_csv("interactions_train.csv")
pp_recipes = pd.read_csv("PP_recipes.csv")
pp_users = pd.read_csv("PP_users.csv")
raw_interactions = pd.read_csv("RAW_interactions.csv")
raw_recipes = pd.read_csv("RAW_recipes.csv")

In [7]:
pp_users.head()

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39
2,2,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...",27,"[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...",27
3,3,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...",1513,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",1513
4,4,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...",376,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...",376


In [61]:
# Creating a dictionary that will be able to link the pp_recipes and pp_users datasets

id_dict = {}

for i in range(len(pp_recipes)):
    id_dict[pp_recipes['i'][i]] = pp_recipes['id'][i]

new_df = pd.DataFrame()
recipes, ratings = [], []

for i in range(len(pp_users)):
    item = ast.literal_eval(pp_users['items'][i])
    rating = ast.literal_eval(pp_users['ratings'][i])

    for j in range(len(item)):
        recipes.append(id_dict[item[j]])
        ratings.append(rating[j])

# Creating a new dataframe that consists of recipe IDs and ratings

new_df['recipe'] = recipes
new_df['rating'] = ratings

most_rated = list(dict(new_df['recipe'].value_counts()[0:50000]).keys())

# Creating a new dataframe that consists of only the most rated recipes with an average rating of
# 3 or more

new_df2 = new_df[new_df['recipe'].isin(most_rated)]
new_df2 = new_df2.reset_index(drop=True)
new_df2 = new_df2.groupby(['recipe']).mean()
new_df2 = new_df2.reset_index(drop=False)
new_df2 = new_df2[new_df2['rating'] > 3]

raw_recipes2 = raw_recipes[raw_recipes['id'].isin(list(new_df2['recipe']))]
raw_recipes2 = raw_recipes2.reset_index(drop=True)

new_df2 = new_df2.reset_index(drop=True)

In [68]:
# Adding nutrition, time to cook, ingredient, and tag information to new_df2

nutrition, minutes, ingreds, tags = {},{},{},{}

for i in range(len(raw_recipes2)):
    recipe = raw_recipes2['id'][i]
    key_words = ast.literal_eval(raw_recipes2['tags'][i])
    ingredients = ast.literal_eval(raw_recipes2['ingredients'][i])
    nutr = raw_recipes2['nutrition'][i]
    mins = raw_recipes2['minutes'][i]

    
    nutrition[recipe] = nutr
    minutes[recipe] = mins
    ingreds[recipe] = ingredients
    tags[recipe] = key_words

nutrition2, minutes2, ingreds2, tags2 = [],[],[],[]

for j in range(len(new_df2)):
    recipe = new_df2['recipe'][j]

    tags2.append(tags[recipe])
    nutrition2.append(nutrition[recipe])
    minutes2.append(minutes[recipe])
    ingreds2.append(ingreds[recipe])


new_df2['tags'] = tags2
new_df2['nutrition'] = nutrition2
new_df2['minutes'] = minutes2
new_df2['ingredients'] = ingreds2


In [69]:
new_df2.head()

,recipe,rating,tags,nutrition,minutes,ingredients
0,40,4.800000,"[60-minutes-or-less, time-to-make, course, pre...","[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",35,"[sugar, lemons, rind of, fresh water, fresh le..."
1,49,4.000000,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch..."
2,58,4.333333,"[weeknight, time-to-make, course, main-ingredi...","[280.1, 9.0, 36.0, 24.0, 59.0, 7.0, 7.0]",164,"[beef eye round, vegetable oil, dried thyme le..."
3,62,4.666667,"[30-minutes-or-less, time-to-make, course, mai...","[407.8, 23.0, 17.0, 0.0, 34.0, 11.0, 18.0]",25,"[fresh lemon juice, olive oil, black beans, fr..."
4,66,4.933333,"[lactose, 30-minutes-or-less, time-to-make, co...","[772.0, 6.0, 657.0, 93.0, 13.0, 2.0, 63.0]",30,"[brewed coffee, ketchup, red wine vinegar, dar..."


In [70]:
# Droping the rating column

new_df2 = new_df2.drop('rating',axis=1)  

In [71]:
# Making a copy of new_df2 and labeling it final_df

final_df = new_df2.copy()
final_df.head() 

,recipe,tags,nutrition,minutes,ingredients
0,40,"[60-minutes-or-less, time-to-make, course, pre...","[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",35,"[sugar, lemons, rind of, fresh water, fresh le..."
1,49,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch..."
2,58,"[weeknight, time-to-make, course, main-ingredi...","[280.1, 9.0, 36.0, 24.0, 59.0, 7.0, 7.0]",164,"[beef eye round, vegetable oil, dried thyme le..."
3,62,"[30-minutes-or-less, time-to-make, course, mai...","[407.8, 23.0, 17.0, 0.0, 34.0, 11.0, 18.0]",25,"[fresh lemon juice, olive oil, black beans, fr..."
4,66,"[lactose, 30-minutes-or-less, time-to-make, co...","[772.0, 6.0, 657.0, 93.0, 13.0, 2.0, 63.0]",30,"[brewed coffee, ketchup, red wine vinegar, dar..."


In [72]:
# The ingredients that will be available for the user to select on the front end will be 
# the 1100 most frequent ingredients listed in the data. The list of ingredients is saved as a picke file
# so that it may be loaded into the django code and displayed on the front end.

ings = []

for i in range(len(final_df)):
    ings.extend(final_df['ingredients'][i])

cnt = Counter(ings)
ingredient_list = sorted(cnt, key=cnt.get, reverse=True)[0:1100]

with open(r'D:\recipe_recommender\recipe_recommender_webiste\recommender_app\static\ingredients2.pickle', 'wb') as f: 
    pickle.dump(ingredient_list, f)

In [73]:
# The ethnicity, dietary restrictions, types of food, and holidays that a user will be able to select from. 
# Also a list named delete which will contain values that will be removed from the final dataset

ethnicity = ['spanish','egyptian','tex-mex','welsh','australian','polynesian','pacific-northwest',
'swiss','italian','german','sudanese','chilean','costa-rican','hawaiian','malaysian','icelandic','norwegian',
'pennsylvania-dutch','somalian','pakistani','nepalese','venezuelan','asian','puerto-rican','nigerian',
'ethiopian','vietnamese','hungarian','laotian','dutch','chinese','south-african','brazilian','austrian',
'greek','middle-eastern','iraqi','ecuadorean','creole','african','korean','cambodian','caribbean','thai',
'indonesian','south-american','midwestern','north-american','mongolian','libyan','new-zealand','russian',
'congolese','mexican','south-west-pacific','japanese','native-american','portuguese','palestinian','cuban',
'scottish','central-american','filipino','indian','honduran','european','saudi-arabian','guatemalan','argentine',
'irish','turkish','georgian','british-columbian','peruvian','scandinavian','southern-united-states','lebanese',
'czech','canadian','cajun','northeastern-united-states','colombian','southwestern-united-states']

delete = ['cocktails','smoothies','beverages'] 

dietary_restrict = ['vegan','vegetarian','dairy-free','gluten-free','nut-free']

types_of_food = ['spreads','omelets-and-frittatas','frozen-desserts','dips','salsas',
'shakes','appetizers','sandwiches','cobblers-and-crisps','soul',
'seafood','pies','salads','barbecue','stews','side-dishes']

holiday = ['super-bowl','rosh-hashanah','new-years','mothers-day','st-patricks-day',
'fathers-day','valentines-day','ramadan','chinese-new-year','easter','christmas',
'april-fools-day','cinco-de-mayo','independence-day','labor-day','memorial-day',
'passover','kwanzaa','thanksgiving','hanukkah','halloween']


In [74]:
# Looping through the dataset and creating ethnicity, type of food, dietary restriction, and holiday
# columns by extracting information from the tags

delete_recipes,ethnicity_lst,type_of_food_lst,holiday_lst,dr_lst = [],[],[],[],[]

for i in range(len(final_df)):
    tags = final_df['tags'][i]

    ethnicity_check = False
    type_of_food_check = False
    holiday_check = False
    dr_check = False

    ethnicity_lst2,type_of_food_lst2,holiday_lst2,dr_lst2 = [],[],[],[]

    for j in range(len(tags)):

        if tags[j] in delete:
            delete_recipes.append(final_df['recipe'][i])
            
        if tags[j] in ethnicity:
            ethnicity_lst2.append(tags[j])
            ethnicity_check = True

        if tags[j] in types_of_food:
            type_of_food_lst2.append(tags[j])
            type_of_food_check = True

        if tags[j] in holiday:
            holiday_lst2.append(tags[j])
            holiday_check = True

        if tags[j] in dietary_restrict:
            dr_lst2.append(tags[j])
            dr_lst_check = True


    if ethnicity_check == False:
        ethnicity_lst2.append("N/A")
    
    if type_of_food_check == False:
        type_of_food_lst2.append("N/A")

    if holiday_check == False:
        holiday_lst2.append("N/A")

    if dr_check == False:
        dr_lst2.append("N/A")

    ethnicity_lst.append(ethnicity_lst2)
    type_of_food_lst.append(type_of_food_lst2)
    holiday_lst.append(holiday_lst2)
    dr_lst.append(dr_lst2)


final_df['ethnicity'] = ethnicity_lst
final_df['type_of_food'] = type_of_food_lst
final_df['holiday'] = holiday_lst
final_df['dietary_restriction'] = dr_lst

final_df = final_df[~final_df['recipe'].isin(delete_recipes)]
final_df = final_df.reset_index(drop=True)

In [75]:
# Creating a column named ingredients bin which will be needed for the recommendation 
# algorithm

ingredients_bin = []

for i in range(len(final_df['ingredients'])):
    binaryList = []
    for ingred in ingredient_list:
        if ingred in final_df['ingredients'][i]:
            binaryList.append(1)
        else:
            binaryList.append(0)
            
    ingredients_bin.append(binaryList)
    

final_df['ingredients_bin'] = ingredients_bin
final_df = final_df.set_index("recipe")
final_df['nutrition'] = final_df.nutrition.apply(lambda x: literal_eval(x))
final_df = final_df[final_df['minutes'] < 3000]


In [77]:
# Creating columns representing the nutritional information

cals,sug,prot,car = [],[],[],[]
nutrition_lst = list(final_df['nutrition'])

for i in range(len(nutrition_lst)):
    cals.append(nutrition_lst[i][0])
    sug.append(nutrition_lst[i][2])
    prot.append(nutrition_lst[i][4])
    car.append(nutrition_lst[i][6])

final_df['calories'] = cals
final_df['sugar'] = sug
final_df['protein'] = prot
final_df['carbs'] = car

In [78]:
# Creating columns to hold the recipe steps, description, and recipe name

recipe_names = {}
recipe_steps = {}
recipe_description = {}
for i in range(len(raw_recipes2)):
    recipe_names[raw_recipes2['id'][i]] = raw_recipes2['name'][i]
    recipe_steps[raw_recipes2['id'][i]] = raw_recipes2['steps'][i]
    recipe_description[raw_recipes2['id'][i]] = raw_recipes2['description'][i]


final_df = final_df.reset_index()
recipe_name,recipe_step,recipe_desc = [],[],[]
for i in range(len(final_df)):
    recipe_name.append(recipe_names[final_df['recipe'][i]])

for j in range(len(final_df)):
    recipe_step.append(recipe_steps[final_df['recipe'][j]])

for k in range(len(final_df)):
    recipe_desc.append(recipe_description[final_df['recipe'][k]])


final_df['recipe_name'] = recipe_name
final_df['recipe_steps'] = recipe_step
final_df['description'] = recipe_desc

final_df = final_df.set_index('recipe')

In [25]:
# This cell converts the ingredients, steps, description, and recipe name columns into a format that can be
# displayed on the front end

ingredients2,recipe_steps2,description2,recipe_name2 = [],[],[],[]
final_df = final_df.reset_index()

for i in range(len(final_df)):
    ings = [j.title() for j in final_df['ingredients'][i]]
    ings2 = ", ".join(ings)
    ingredients2.append(ings2)

    steps = ast.literal_eval(final_df['recipe_steps'][i])
    steps2 = [j.title() for j in steps]
    steps3 = ", ".join(steps2)
    recipe_steps2.append(steps3)

    desc = str(final_df['description'][i]).title()
    description2.append(desc)

    recipe_name = str(final_df['recipe_name'][i]).title()
    recipe_name2.append(recipe_name)


final_df['ingredients_html'] = ingredients2
final_df['steps_html'] = recipe_steps2
final_df['description_html'] = description2
final_df['recipe_name_html'] = recipe_name2

In [27]:
# Creating a tag bins column which will be needed for the recommendation algorithm

tgs = []

for i in range(len(final_df)):
    tgs.extend(final_df['tags'][i])

cnt = Counter(tgs)
tag_list = sorted(cnt, key=cnt.get, reverse=True)

tags_bin = []

for i in range(len(final_df['tags'])):
    binaryList = []
    for tag in tag_list:
        if tag in final_df['tags'][i]:
            binaryList.append(1)
        else:
            binaryList.append(0)
            
    tags_bin.append(binaryList)


final_df['tag_bins'] = tags_bin

In [14]:
# Creating three dictionaries which map all distinct ingredients, ethnicities, and dietary restrictions
# to recipe IDs for faster lookup times in the django code. 

ingredient_dict = defaultdict(list)
ethnicity_dict = defaultdict(list)
dietary_restriction_dict = defaultdict(list)


df_i = final_df.explode('ingredients')
df_i = df_i.reset_index(drop=True)

for i in range(len(df_i)):
    ingredient_dict[df_i['ingredients'][i]].append(df_i['recipe'][i])

df_e = final_df.explode('ethnicity')
df_e = df_e.reset_index(drop=True)

for i in range(len(df_e)):
    ethnicity_dict[df_e['ethnicity'][i]].append(df_e['recipe'][i])


df_d = final_df.explode('dietary_restriction')
df_d = df_d.reset_index(drop=True)

for i in range(len(df_d)):
    dietary_restriction_dict[df_d['dietary_restriction'][i]].append(df_d['recipe'][i])

In [31]:
# All dictionaries are saved as pickle files to be used in the django code


with open(r"D:\recipe_recommender\recipe_recommender_webiste\recommender_app\static\ingredient_map.pkl", 'wb') as handle:
    pickle.dump(ingredient_dict, handle)

with open(r"D:\recipe_recommender\recipe_recommender_webiste\recommender_app\static\ethnicity_map.pkl", 'wb') as handle:
    pickle.dump(ethnicity_dict, handle)

with open(r"D:\recipe_recommender\recipe_recommender_webiste\recommender_app\static\dr_map.pkl", 'wb') as handle:
    pickle.dump(dietary_restriction_dict, handle)

In [18]:
# Saving the final dataset as an excel file

final_df.to_excel("recipe_data2.xlsx",index=False)